In [14]:
! pip install geopandas
! pip install pulp
! pip install spopt
! pip install coinor-cbc
! pip install matplotlib-scalebar

ERROR: Could not find a version that satisfies the requirement coinor-cbc (from versions: none)
ERROR: No matching distribution found for coinor-cbc


In [15]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-07-04T17:20:57.492587+07:00

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [16]:
import pandas as pd
import networkx as nx
import geopandas
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.lines as mlines
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point
import time
import numpy as np
import pulp
import shapely
import spopt
from spopt.locate import PCenter, simulated_geo_points

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # ignore deprecation warning - GH pysal/spaghetti#649
    import spaghetti

%watermark -w
%watermark -iv

Watermark: 2.4.3

matplotlib         : 3.9.0
pulp               : 2.8.0
spaghetti          : 1.7.5.post1
shapely            : 2.0.4
numpy              : 1.26.4
pandas             : 2.2.2
spopt              : 0.6.0
networkx           : 3.3
matplotlib_scalebar: 0.8.1
geopandas          : 0.14.4



In [17]:
data = pd.read_csv('C:/Users/ASUS/Downloads/rescue_station_project/data/csv/Thuyhe_HCM_motphan_test_graph.csv')
# data.head(6)
data

,FID_Thuyhe,Id,FromPoint,ToPoint,chieudai
0,1,1,0,1,10.000000
1,1,2,1,2,10.000000
2,1,3,2,3,10.000000
3,1,4,3,4,10.000000
4,1,5,4,5,10.000000
...,...,...,...,...,...
9840,1,9841,9841,9842,10.000000
9841,1,9842,9842,9843,10.000000
9842,1,9843,9843,9844,10.000000
9843,1,9844,9844,9845,5.120352


In [18]:
df_rows = data.iloc[:9845]

# Hiển thị kết quả
print(df_rows)

      FID_Thuyhe    Id  FromPoint  ToPoint   chieudai
0              1     1          0        1  10.000000
1              1     2          1        2  10.000000
2              1     3          2        3  10.000000
3              1     4          3        4  10.000000
4              1     5          4        5  10.000000
...          ...   ...        ...      ...        ...
9840           1  9841       9841     9842  10.000000
9841           1  9842       9842     9843  10.000000
9842           1  9843       9843     9844  10.000000
9843           1  9844       9844     9845   5.120352
9844           1  9845       9845     7119  10.000000

[9845 rows x 5 columns]


In [19]:
# Chọn 3 cột và sắp xếp lại thứ tự cột
df_selected = df_rows[['Id', 'FromPoint', 'ToPoint', 'chieudai']]

# Đổi tên các cột
#df_selected.columns = ['distance', 'name', 'DestinationName']

# Hiển thị kết quả
print(df_selected)

        Id  FromPoint  ToPoint   chieudai
0        1          0        1  10.000000
1        2          1        2  10.000000
2        3          2        3  10.000000
3        4          3        4  10.000000
4        5          4        5  10.000000
...    ...        ...      ...        ...
9840  9841       9841     9842  10.000000
9841  9842       9842     9843  10.000000
9842  9843       9843     9844  10.000000
9843  9844       9844     9845   5.120352
9844  9845       9845     7119  10.000000

[9845 rows x 4 columns]


In [20]:
# Nhập id x và id y từ người dùng
x = int(input("Nhập id x: "))
y = int(input("Nhập id y: "))

# Lọc ra các dòng có Id nằm trong khoảng từ id x đến id y
filtered_df = df_selected[(df_selected['Id'] >= x) & (df_selected['Id'] <= y)]

# In ra DataFrame mới
print(filtered_df)


#note 1 => chỉnh code cho phép nhập id để chọn ra vùng cần tính
#note 2 => cho chọn cạnh trực tiếp trên hình, lọc id để chọn ra vùng cần tính

      Id  FromPoint  ToPoint  chieudai
99   100         99      100      10.0
100  101        100      101      10.0
101  102        101      102      10.0
102  103        102      103      10.0
103  104        103      104      10.0
104  105        104      105      10.0
105  106        105      106      10.0
106  107        106      107      10.0
107  108        107      108      10.0
108  109        108      109      10.0
109  110        109      110      10.0
110  111        110      111      10.0
111  112        111      112      10.0
112  113        112      113      10.0
113  114        113      114      10.0
114  115        114      115      10.0
115  116        115      116      10.0
116  117        116      117      10.0
117  118        117      118      10.0
118  119        118      119      10.0
119  120        119      120      10.0


In [21]:
# Tạo một bản sao độc lập của DataFrame filtered_df
filtered_df_copy = filtered_df.copy()

# Tính tổng của cột 'chieudai'
net_length_value = filtered_df_copy['chieudai'].sum()

# Gán giá trị của tổng vào cột 'net_length' của hàng đầu tiên
filtered_df_copy.loc[filtered_df_copy.index[0], 'net_length'] = net_length_value

# Gán giá trị NaN cho các hàng còn lại của cột 'net_length'
filtered_df_copy.loc[filtered_df_copy.index[1:], 'net_length'] = float('0')

# In ra DataFrame mới có cột 'net_length'
print(filtered_df_copy)


      Id  FromPoint  ToPoint  chieudai  net_length
99   100         99      100      10.0       210.0
100  101        100      101      10.0         0.0
101  102        101      102      10.0         0.0
102  103        102      103      10.0         0.0
103  104        103      104      10.0         0.0
104  105        104      105      10.0         0.0
105  106        105      106      10.0         0.0
106  107        106      107      10.0         0.0
107  108        107      108      10.0         0.0
108  109        108      109      10.0         0.0
109  110        109      110      10.0         0.0
110  111        110      111      10.0         0.0
111  112        111      112      10.0         0.0
112  113        112      113      10.0         0.0
113  114        113      114      10.0         0.0
114  115        114      115      10.0         0.0
115  116        115      116      10.0         0.0
116  117        116      117      10.0         0.0
117  118        117      118   

In [22]:
# Tạo đồ thị từ dataframe
G = nx.from_pandas_edgelist(filtered_df, 'FromPoint', 'ToPoint', ['chieudai'])

# Tính toán khoảng cách giữa các cặp đỉnh
distances = []

for source in filtered_df['FromPoint']:
    for target in filtered_df['ToPoint']:
        if source != target:
            distance = nx.shortest_path_length(G, source=source, target=target, weight='chieudai')
            distances.append({'FromPoint': source, 'ToPoint': target, 'chieudai': distance})

# Tạo dataframe từ kết quả
distance_df = pd.DataFrame(distances)

print(distance_df)


     FromPoint  ToPoint  chieudai
0           99      100      10.0
1           99      101      20.0
2           99      102      30.0
3           99      103      40.0
4           99      104      50.0
..         ...      ...       ...
416        119      115      40.0
417        119      116      30.0
418        119      117      20.0
419        119      118      10.0
420        119      120      10.0

[421 rows x 3 columns]


In [23]:
# Trộn các kết quả vào chung 3 cột chieudai, FromPoint, ToPoint
merged_df = pd.concat([filtered_df[['FromPoint', 'ToPoint', 'chieudai']], distance_df])

# Reset index để tránh index trùng lặp
merged_df.reset_index(drop=True, inplace=True)

# Hiển thị kết quả
print(merged_df)


     FromPoint  ToPoint  chieudai
0           99      100      10.0
1          100      101      10.0
2          101      102      10.0
3          102      103      10.0
4          103      104      10.0
..         ...      ...       ...
437        119      115      40.0
438        119      116      30.0
439        119      117      20.0
440        119      118      10.0
441        119      120      10.0

[442 rows x 3 columns]


In [24]:
# Đổi tên các cột
df = merged_df.rename(columns={"FromPoint": "StartPoint", "ToPoint": "EndPoint", "chieudai": "Distance"})

print(df)

     StartPoint  EndPoint  Distance
0            99       100      10.0
1           100       101      10.0
2           101       102      10.0
3           102       103      10.0
4           103       104      10.0
..          ...       ...       ...
437         119       115      40.0
438         119       116      30.0
439         119       117      20.0
440         119       118      10.0
441         119       120      10.0

[442 rows x 3 columns]


In [25]:
# Thêm cột 'net_length' vào DataFrame 2
df['net_length'] = 0.0  # Khởi tạo cột với giá trị 0
df.loc[0, 'net_length'] = net_length_value  # Gán giá trị của tổng vào hàng đầu tiên

# In ra DataFrame mới có cột 'net_length'
print(df)

     StartPoint  EndPoint  Distance  net_length
0            99       100      10.0       210.0
1           100       101      10.0         0.0
2           101       102      10.0         0.0
3           102       103      10.0         0.0
4           103       104      10.0         0.0
..          ...       ...       ...         ...
437         119       115      40.0         0.0
438         119       116      30.0         0.0
439         119       117      20.0         0.0
440         119       118      10.0         0.0
441         119       120      10.0         0.0

[442 rows x 4 columns]


In [26]:
import os

# Đường dẫn đến thư mục
folder_path = r'C:/Users/ASUS/Downloads/rescue_station_project/data/csv'

# Kiểm tra nếu thư mục không tồn tại, tạo mới
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Tạo đường dẫn đến file CSV trong thư mục
file_path = os.path.join(folder_path, 'network_distance.csv')

# Lưu dataframe thành file CSV
df.to_csv(file_path, index=False)

# Hiển thị kết quả
print("File CSV đã được lưu thành công vào thư mục:", folder_path)

File CSV đã được lưu thành công vào thư mục: C:/Users/ASUS/Downloads/rescue_station_project/data/csv
